In [ ]:
# Start program
programName = "Monthly PUC Reporting"

In [ ]:
# Setup Environment

import pandas as pd


# os for listing files
import os

from dotenv import load_dotenv
# for streaming envrionment variables
from io import StringIO

envFile = open('.env','r')

load_dotenv(stream=envFile)

# for moving files
import shutil

# for datetime
import datetime

# For documenting
import json

# Import shared libraries
import sys
sys.path.append(os.getenv('SHARED_LIBRARIES'))

# SQL Alchemy for database connections
from sqlalchemy.engine import create_engine
import sqlalchemy as sa

# Logging
from SharedLogger import createLogger

# for reading emails
from O365Manager import sendEmail

programDirectory = os.getcwd()

fileSuffix = datetime.datetime.now().strftime("%Y-%m-%d")
logger = createLogger(programDirectory+'\\log\\',programName + ' - ' + fileSuffix + '.log')

In [ ]:
# Secrets from Keeper

from keeper_secrets_manager_core import SecretsManager
from keeper_secrets_manager_core.storage import FileKeyValueStorage

secrets_manager = SecretsManager(
    config=FileKeyValueStorage(os.getenv('KSM_CONFIG'))
)

ids = [os.getenv('KEEPER_ID')]

record = secrets_manager.get_secrets(ids)[0]

database=record.custom_field('Database',single=True)
user=record.field('login',single=True)
password=record.field('password',single=True)

In [ ]:
# Connect to Database

logger.info("Connecting to " + database + ' Database')

connectionLive = "oracle+cx_oracle://" + user + ":" + password + '@' + database

engineLive = create_engine(connectionLive)

In [ ]:
# Create and Send file Function

def sendFile(dl,query,fileName,subject,body):

    logger.info(subject)

    data = pd.read_sql(query,engineLive)

    logger.info(str(len(data)) + ' records, send email!')

    file = programDirectory+ '\\output\\' + fileName + ' - ' + fileSuffix + '.xlsx'

    data.to_excel(file,index=False)

    sendEmail(dl,subject,"You have " + str(len(data)) + " records to check! \n\n " + body,**{
        "attachment":file
    })




In [ ]:
# CORR adjustments

sql = """
SELECT distinct
   AJ.ACCOUNT_NO
  ,PE.PERSON_NO
  ,PE.PERSON_LNM
  ,PE.PERSON_FNM
  ,AJ.ITEM_DATE
  ,AJ.ITEM_TYPE
  ,AJ.ADD_BY
  ,AJ.ACCOUNT_ADJ_COUNT
  ,AJ.ACCOUNT_ADJ_TOTAL_AMT
  ,AG.ALL_ADJ_COUNT
  ,AC.CYCLE_CD  --added RHS
  --,AG.ALL_ADJ_TOTAL_AMT
                --begin RHS
  ,lo.house_no
  ,lo.street_pfx_dir
  ,lo.street_nm
  ,lo.street_nm_sfx
  ,lo.street_sfx_dir
  ,lo.sec_addr_id
  ,lo.sec_addr_range
  ,lo.city
  ,lo.province_cd
  ,lo.postal_code
  , cc.add_by
  , cc.add_dtm
  , cc.contact_desc
  , bcc.add_by
  , bcc.add_dtm
  , bcc.contact_desc
                --end RHS
FROM
                          (SELECT  SS_ACCOUNT_NO                              as ACCOUNT_NO
                                  ,item_dt                                    as ITEM_DATE
                                  ,item_tp                                    as ITEM_TYPE
                                  ,add_by                                     as ADD_BY
                                  ,SUM((ITEM_AMT))                            as ACCOUNT_ADJ_TOTAL_AMT
                                  ,COUNT(*)                                   as ACCOUNT_ADJ_COUNT
                                  ,ROW_NUMBER() OVER (ORDER BY SS_ACCOUNT_NO) as ACCOUNT_ROW_NUMBER
                           FROM   CAYENTA.AR00200T
                           WHERE  SUBSYSTEM_ID = 'UM'
                             AND  SS_ACCOUNT_NO > '/'
                             AND  ITEM_DT BETWEEN LAST_DAY(ADD_MONTHS(TRUNC(SYSDATE),-2))+1
                                              AND LAST_DAY(ADD_MONTHS(TRUNC(SYSDATE),-1))
                             AND  nvl(RTRIM(ADJ_REASON_CD),' ') = 'CORR' --NOT IN ('BALX','PINE')
                             AND  SOURCE_REFERENCE = 'ARHDR'
                             AND ITEM_TP = 'MANADJ' -- ADDED BY MIOBST 20190515 TO REMOVE CANCEL/REBILLS
                            GROUP BY SS_ACCOUNT_NO, item_dt, item_tp, add_by
                          )
                      AJ
JOIN CAYENTA.UM00200M AC  ON AJ.ACCOUNT_NO    = AC.ACCOUNT_NO
left outer join cayenta.UM00252V RL  on AC.ACCOUNT_NO    = RL.ACCOUNT_NO  --new RHS
left outer join cayenta.UM00100M lo  on RL.LOCATION_NO   = LO.LOCATION_NO --new RHS
JOIN CAYENTA.SG00100M PE  ON AC.PERSON_NO     = PE.PERSON_NO
JOIN                      (SELECT SUM(ABS(ITEM_AMT))                         as ALL_ADJ_TOTAL_AMT
                                 ,COUNT(*)                                   as ALL_ADJ_COUNT
                           FROM   CAYENTA.AR00200T
                           WHERE  SUBSYSTEM_ID = 'UM'
                             AND  SS_ACCOUNT_NO > '/'
                             AND  ITEM_DT BETWEEN LAST_DAY(ADD_MONTHS(TRUNC(SYSDATE),-2))+1
                                              AND LAST_DAY(ADD_MONTHS(TRUNC(SYSDATE),-1))
                             AND  nvl(RTRIM(ADJ_REASON_CD),' ') = 'CORR' --NOT IN ('BALX','PINE')
                             AND  SOURCE_REFERENCE = 'ARHDR'
                             AND ITEM_TP = 'MANADJ' -- ADDED BY MIOBST 20190515 TO REMOVE CANCEL/REBILLS
                          )
                      AG   ON 1=1
--begin RHS
    left outer join (
                     select cs.contact_no, cs.ownership_cd, cs.ownership_key, cs.add_by, cs.add_dtm,
                         cd.contact_desc
                      from cayenta.sg00810t cs
                        join cayenta.sg00800t cd on cs.contact_no = cd.contact_no and cd.contact_tp = 'ADJST'
                      where cs.ownership_cd = 'ACCT'
                        )
                        cc on cc.ownership_key = aj.account_no --to_char (ac.ACCOUNT_NO)
                         and trunc(cc.add_dtm) = to_date(trunc(AJ.ITEM_DATE))
    left outer join (
                     select cs.contact_no, cs.ownership_cd, cs.ownership_key, cs.add_by, cs.add_dtm,
                         cd.contact_desc
                      from cayenta.sg00810t cs
                        join cayenta.sg00800t cd on cs.contact_no = cd.contact_no and cd.contact_tp = 'BILL'
                      where cs.ownership_cd = 'ACCT'
                    )
                    bcc on bcc.ownership_key = to_char (AJ.ACCOUNT_NO)
                      and trunc(bcc.add_dtm) = to_date(trunc(AJ.ITEM_DATE),'DD-MON-YYYY')
--end RHS
"""

sendFile(os.getenv('dl').split(','),sql,"CORR Adjustments","[PUC REPORTING] CORR Adjustments Report","Please find your CORR Adjustments report attached!")

In [ ]:
# Cancel/Rebill

sql = """
SELECT distinct bh.account_no
  , bh.cust_first_name
--  , bh.cust_middle_name  removed RHS
  , bh.cust_last_name
  , als.location_no
  , l.house_no
  , l.street_pfx_dir
  , l.street_nm
  , l.street_nm_sfx
  , l.street_sfx_dir
  , l.city
  , l.province_cd
  , l.postal_code
  , ac.cycle_cd        --added RHS
  , bh.bill_tp
  , ar.adj_reason_cd
  , bh.invoice_no
  , bh.bill_dt
  , bh.bill_generated_chgs + bh.non_bill_generated_chgs AS BILL_AMT
  , bh.period_from_dt
  , bh.period_to_dt
  , bh.canceled_bill_yn
  , bh.corrected_bill_yn
  , bh.rebill_bill_yn
  , bh.chg_dtm
  , bh.chg_by
  , ar.item_dt
               --begin RHS
  , cc.add_by
  , cc.add_dtm
  , cc.contact_desc
  , bcc.add_by
  , bcc.add_dtm
  , bcc.contact_desc
               --end RHS
  FROM cayenta.um00260t bh
    JOIN cayenta.um00250t als ON als.account_no = bh.account_no
    join cayenta.um00200m ac on ac.account_no = bh.account_no  --new RHS
    JOIN cayenta.ar00200t ar ON ar.ss_account_no = bh.account_no
    JOIN cayenta.um00100m l ON l.location_no = als.location_no
--begin add RHS
    left outer join (
                     select cs.contact_no, cs.ownership_cd, cs.ownership_key, cs.add_by, cs.add_dtm,
                         cd.contact_desc
                      from cayenta.sg00810t cs
                        join cayenta.sg00800t cd on cs.contact_no = cd.contact_no and cd.contact_tp = 'ADJST'
                      where cs.ownership_cd = 'ACCT'
                     )
                     cc on cc.ownership_key = to_char (bh.ACCOUNT_NO)
                       and trunc(cc.add_dtm) = to_date(trunc(AR.ITEM_DT))
    left outer join (
                     select cs.contact_no, cs.ownership_cd, cs.ownership_key, cs.add_by, cs.add_dtm,
                         cd.contact_desc
                      from cayenta.sg00810t cs
                        join cayenta.sg00800t cd on cs.contact_no = cd.contact_no and cd.contact_tp = 'BILL'
                      where cs.ownership_cd = 'ACCT'
                     )
                     bcc on bcc.ownership_key = to_char (bh.ACCOUNT_NO)
                        and trunc(bcc.add_dtm) = to_date(trunc(AR.ITEM_DT))
--end add by RHS
  WHERE ((bh.canceled_bill_yn = 'T' AND ar.ss_bill_dt = bh.bill_dt) OR (bh.rebill_bill_yn = 'T' AND bh.canceled_bill_yn = 'F'))
    AND bh.audit_or_live != 'A'
    AND bh.error_yn = 'F'
    -- for manually setting date range
    -- AND bh.chg_dtm >= '20181201'
    -- AND bh.chg_dtm < '20190101'
    AND  bh.chg_dtm BETWEEN LAST_DAY(ADD_MONTHS(TRUNC(SYSDATE),-2))+1
    AND LAST_DAY(ADD_MONTHS(TRUNC(SYSDATE),-1))
    AND ar.adj_reason_cd IN ('CORR', 'CNRB', 'CORE')
    AND ar.source_spec2 = 'CANRB'
ORDER BY adj_reason_cd, account_no, invoice_no
"""

sendFile(os.getenv('dl').split(','),sql,"Cancel Rebill","[PUC REPORTING] Cancel/Rebill Report","Please find your Cancel/Rebill report attached!")

In [ ]:
# Bill Estimates

sql = """
select
summary.*,
case when estimates.consecutive_estimates is null then 0 else estimates.consecutive_estimates end as consecutive_estimates
from (
    select 
    to_char(BILL_RUN_DT,'YYYY') as BILL_YEAR,
    to_char(BILL_RUN_DT,'MM') as BILL_MONTH,
    account_no,
    count(invoice_no) as total_services,
    sum(case when read_est_yn = 'T' then 1 else 0 end) as read_est_num
    from cayenta.BI_BILL_XML_METER_DETAILS D
    where
    BILL_RUN_DT BETWEEN LAST_DAY(ADD_MONTHS(TRUNC(SYSDATE),-2))+1 AND LAST_DAY(ADD_MONTHS(TRUNC(SYSDATE),-1))
    group by to_char(BILL_RUN_DT,'YYYY'), to_char(BILL_RUN_DT,'MM'), account_no
) summary 

/*
    Get total consecutive estimates for estimated accounts
*/

left join (
    select
    account_no,
    sum(last_read_est) as consecutive_estimates
    from (
    select 
    D.account_no,
    D.register_tp,
    bill_run_dt,
    read_est_yn,
    case when lag(read_est_yn) over (order by D.account_no, D.register_tp, bill_run_dt) = read_est_yn and read_est_yn = 'T' then 1 else 0 end as last_read_est
    from cayenta.BI_BILL_XML_METER_DETAILS D
    
    /* Get those estimated within this timeframe */
    inner join (
        select
        distinct
        account_no,
        register_tp
        from cayenta.BI_BILL_XML_METER_DETAILS D
        where
        BILL_RUN_DT BETWEEN LAST_DAY(ADD_MONTHS(TRUNC(SYSDATE),-2))+1 AND LAST_DAY(ADD_MONTHS(TRUNC(SYSDATE),-1))
        and
        read_est_yn = 'T'
    ) est
    on est.account_no = D.account_no and est.register_tp = D.register_tp
    order by account_no, register_tp, bill_run_dt desc
    ) a
    group by account_no
) estimates
on summary.account_no = estimates.account_no
"""

sendFile(os.getenv('dl').split(','),sql,"Estimated Reads","[PUC REPORTING] Estimated Reads Report","Please find your Estimated Reads report attached!")

In [ ]:
# Bills over 10 days after bill date

sql = """
WITH late_bills AS (
SELECT DISTINCT T.invoice_no, T.account_no, T.bill_run_dt, T.bill_run_tm, T.bill_dt, T.period_from_dt, T.period_to_dt, T.billing_days, T.final_yn, T.bill_tp, T.add_dtm, T.off_cycle_yn, T.canceled_bill_yn
,O.bill_trxn_no, R.cycle_cd, R.sched_extract_dtm, R.billed_dtm
,CASE
    WHEN O.bill_trxn_no IS NULL 
      THEN T.bill_run_dt - period_to_dt --Nonmetered services only
    WHEN NVL(TRUNC(R.billed_dtm),TO_DATE('20991231','yyyymmdd')) > T.bill_run_dt AND T.billing_days > 40
      THEN T.billing_days - 30 --Bill trxn no is associated to a billing after the bill's date, look at these if the billing period is long
    WHEN NVL(TRUNC(R.billed_dtm),TO_DATE('20991231','yyyymmdd')) > T.bill_run_dt AND T.billing_days <= 40
      THEN T.bill_run_dt - period_to_dt --Bill trxn no is associated to a billing after the bill's date, if the bill isn't long, evaluate how long after the period_to_dt it billed   
  ELSE T.bill_run_dt - TRUNC(R.billed_dtm)
  END AS days_late
FROM cayenta.um00260t T --bill header table
LEFT OUTER JOIN cayenta.um00163t O ON T.account_no = O.account_no --join read table to get period_trxn_no to link meter bills back to the extract schedule
             AND (TRUNC(T.bill_run_dt) = TRUNC(O.billed_dtm) AND (to_char(T.bill_run_tm, 'hh:mi:ss AM') = to_char(O.billed_dtm, 'hh:mi:ss AM')) )
             AND O.read_tp NOT IN ('CUTON','SET','PULL')
LEFT OUTER JOIN cayenta.um50700c R on O.bill_trxn_no = R.period_trxn_no --extract schedule
WHERE T.audit_or_live = 'L'
AND T.error_yn <> 'T'
AND T.bill_tp NOT IN ('COLL','REVRS','FINAL')
AND T.rebill_bill_yn <> 'T'
AND T.canceled_bill_yn <> 'T'
AND T.bill_run_dt - NVL(TRUNC(R.billed_dtm),TO_DATE('20991231','yyyymmdd')) <> 0 --these are perfect accounts according to bill_dt versus bill_dt of bill_trxn_no 
AND T.bill_run_dt between LAST_DAY(ADD_MONTHS(TRUNC(SYSDATE),-2))+1 AND LAST_DAY(ADD_MONTHS(TRUNC(SYSDATE),-1))
)
SELECT * FROM late_bills WHERE days_late > 10
ORDER BY days_late
"""

sendFile(os.getenv('dl').split(','),sql,"Bills over 10 Days","[PUC REPORTING] Bills Over 10 Days","Please find your bills over 10 days report attached!")

In [ ]:
# Finished
logger.info('Program finished!')